# visualization of journeys
(just need to run until the time slider cell)

In [1]:
import folium
import pandas as pd
from ipywidgets import widgets
import IPython.display as disp
import os

In [2]:
routes = pd.read_csv(os.path.join('data','mergeddata_filtered.csv'))
routes_all = pd.read_csv(os.path.join('data','mergeddata.csv'))
centroids = pd.read_csv(os.path.join('data','planningarea_latlon.csv'))

In [3]:
def getStyle(count, difference):
    width = count/1000*8
    if difference < 45:
        col = '#fe9929'
        opacity = 0.5
    elif difference < 60:
        col = '#d95f0e'
        opacity = 0.7
    else:
        col = '#993404'
        opacity = 1
    return [width, col, opacity]

In [4]:
def drawmap(time, filtered=True, origin=None):
    m = folium.Map(location=[1.3521, 103.8198], tiles='CartoDB positron', zoom_start=11)
    m.choropleth('geospatial/planningarea_simplified.json', fill_opacity=0, line_color='grey', line_weight='0.5')

    if filtered:
        routes1 = routes[routes.time == str(time)+':00:00'].sort_values(by='difference')
    else:
        routes1 = routes_all[routes_all.time == str(time)+':00:00'].sort_values(by='difference')
    
    if origin:
        routes1 = routes1[routes1.origin_name == origin]
        
    for i in range(routes1.shape[0]):
        r = routes1.iloc[i]
        width, col, opacity = getStyle(r['total_count'], r['difference'])
        folium.PolyLine(
            locations=[
                (r['origin_lat'], r['origin_lon']), 
                (r['dest_lat'], r['dest_lon'])
            ],
            popup=r['popup'] if filtered else None,
            opacity=opacity,
            weight=width,
            color=col
        ).add_to(m)

    return m

# time slider map

In [5]:
# map with time slider and popups!
# line thickness represents number of trips - thicker means more people
# colour and opacity represents time difference (public transport minus car) - darker means greater diff

slider = widgets.IntSlider(
    min=0,
    max=23,
    value=8,
    layout=widgets.Layout(width='100%'),
    continuous_update=False
)
disp.display(slider)

m = drawmap(8)
disp.display(m, display_id='map')

def on_value_change(change):
    m = drawmap(change['new'])
    disp.update_display(m, display_id='map')

slider.observe(on_value_change, names='value')

IntSlider(value=8, continuous_update=False, layout=Layout(width='100%'), max=23)

## by planning area

In [7]:
# dropdown and slider for plannign area/time
# this works but is abit slow to render. and has no popups

select = widgets.Select(
    options=[None, 'ANG MO KIO', 'BEDOK', 'BUKIT BATOK', 'BOON LAY', 'BUKIT MERAH', 'BUKIT PANJANG', 'BISHAN', 'BUKIT TIMAH', 'CHANGI BAY', 'CENTRAL WATER CATCHMENT', 'CHANGI', 'CHOA CHU KANG', 'CLEMENTI', 'DOWNTOWN CORE', 'GEYLANG', 'HOUGANG', 'JURONG EAST', 'JURONG WEST', 'KALLANG', 'LIM CHU KANG', 'MANDAI', 'MARINA EAST', 'MARINE PARADE', 'MARINA SOUTH', 'MUSEUM', 'NORTH-EASTERN ISLANDS', 'NEWTON', 'NOVENA', 'ORCHARD', 'OUTRAM', 'PUNGGOL', 'PAYA LEBAR', 'PIONEER', 'PASIR RIS', 'QUEENSTOWN', 'ROCHOR', 'RIVER VALLEY', 'SEMBAWANG', 'SENGKANG', 'SERANGOON', 'SOUTHERN ISLANDS', 'SUNGEI KADUT', 'SELETAR', 'SIMPANG', 'SINGAPORE RIVER', 'STRAITS VIEW', 'TENGAH', 'TAMPINES', 'TANGLIN', 'TOA PAYOH', 'TUAS', 'WESTERN WATER CATCHMENT', 'WOODLANDS', 'WESTERN ISLANDS', 'YISHUN'],
    value='ANG MO KIO',
    description='Origin:'
)
disp.display(select)

slider1 = widgets.IntSlider(
    min=0,
    max=23,
    value=9,
    layout=widgets.Layout(width='100%'),
    continuous_update=False
)
disp.display(slider1)

m1 = drawmap(slider1.value, filtered=False, origin=select.value)
disp.display(m1, display_id='map1')

def on_area_change(change):
    m1 = drawmap(slider1.value, filtered=False, origin=change['new'])
    disp.update_display(m1, display_id='map1')

def on_time_change(change):
    m1 = drawmap(change['new'], filtered=False, origin=select.value)
    disp.update_display(m1, display_id='map1')

select.observe(on_area_change, names='value')
slider1.observe(on_time_change, names='value')

Select(description='Origin:', index=1, options=(None, 'ANG MO KIO', 'BEDOK', 'BUKIT BATOK', 'BOON LAY', 'BUKIT MERAH', 'BUKIT PANJANG', 'BISHAN', 'BUKIT TIMAH', 'CHANGI BAY', 'CENTRAL WATER CATCHMENT', 'CHANGI', 'CHOA CHU KANG', 'CLEMENTI', 'DOWNTOWN CORE', 'GEYLANG', 'HOUGANG', 'JURONG EAST', 'JURONG WEST', 'KALLANG', 'LIM CHU KANG', 'MANDAI', 'MARINA EAST', 'MARINE PARADE', 'MARINA SOUTH', 'MUSEUM', 'NORTH-EASTERN ISLANDS', 'NEWTON', 'NOVENA', 'ORCHARD', 'OUTRAM', 'PUNGGOL', 'PAYA LEBAR', 'PIONEER', 'PASIR RIS', 'QUEENSTOWN', 'ROCHOR', 'RIVER VALLEY', 'SEMBAWANG', 'SENGKANG', 'SERANGOON', 'SOUTHERN ISLANDS', 'SUNGEI KADUT', 'SELETAR', 'SIMPANG', 'SINGAPORE RIVER', 'STRAITS VIEW', 'TENGAH', 'TAMPINES', 'TANGLIN', 'TOA PAYOH', 'TUAS', 'WESTERN WATER CATCHMENT', 'WOODLANDS', 'WESTERN ISLANDS', 'YISHUN'), value='ANG MO KIO')

IntSlider(value=9, continuous_update=False, layout=Layout(width='100%'), max=23)

## extra code

In [ ]:
# slider and map for all data with count>40
# this works but is slow to render. and has no popups

slider_all = widgets.IntSlider(
    min=0,
    max=23,
    layout=widgets.Layout(width='100%'),
    continuous_update=False
)
disp.display(slider_all)

m = drawmap(0, filtered=False)
disp.display(m, display_id='map_all')

def on_value_change_all(change):
    m = drawmap(change['new'], filtered=False)
    disp.update_display(m, display_id='map_all')

slider_all.observe(on_value_change_all, names='value')

In [ ]:
# for generating static html files
# for hour in [3,8,13,18,22]:
#     mapp = drawmap(hour)
#     mapp.save('folium_output/map-'+str(hour)+'h.html')
#     mapp = drawmap(hour, filtered=False)
#     mapp.save('folium_output/map-all-'+str(hour)+'h.html')

In [ ]:
# for drawing markers. works but is very slow, so not using
m = folium.Map(location=[1.3521, 103.8198], tiles='CartoDB positron', zoom_start=11)
m.choropleth('geospatial/planningarea_simplified.json', fill_opacity=0, line_color='grey', line_weight='0.5')

for i in range(centroids.shape[0]):
    c = centroids.iloc[i]
    folium.Marker(
        location=(c['lat'], c['lon']),
        popup=folium.Popup(c['Planning Area Name']),
        icon=folium.features.CustomIcon('https://image.flaticon.com/icons/png/512/33/33622.png', icon_size=(14, 14))
    ).add_to(m)

m